<a href="https://colab.research.google.com/github/lfarlima/Project-2/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports and Dependencies 

In [ ]:
#imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from newsapi import NewsApiClient
from dotenv import load_dotenv
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer=SentimentIntensityAnalyzer
import pathlib as path
from dotenv import load_dotenv
load_dotenv()

import yfinance as yf # Stock data
from numpy.random import seed
seed (1)
from tensorflow import random
random.set_seed(2)

from nltk.corpus import stopwords, reuters
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
import re
import matplotlib.pyplot as plt

# Code to download corpora
import nltk
nltk.download('wordnet')
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')


lemmatizer = WordNetLemmatizer()

api_key = "0ef1c61926f54984abcca4338225fd66"

# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

newsapi = NewsApiClient(api_key=api_key)



## Twitter Tweet's sentiment analysis 

In [ ]:
# pip install nest_asyncio

In [ ]:
# pip install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
# import twint
# import nest_asyncio
# nest_asyncio.apply()
# c= twint.Config()
# c.Search= "ETH" or "Ethereum"
# c.Since= "2020-09-29"
# c.Until = '2021-04-04'
# # c.Limit= 1000
# c.Lang= "en"
# c.Store_csv= True
# c.Output= "Search.csv"
# twint.run.Search(c)

In [ ]:
# df = pd.read_csv('Search.csv', encoding="utf-8-sig")
# df= df[["id", "created_at", "tweet", "language"]]
# df=df.loc[df["language"]=="en"]
# df=df.rename(columns={"id": "ID", "created_at": "Date", "tweet": "Tweet"})
# df=df.drop(["language"], axis=1)
# #need to standardize time 
# # df=df.set_index("Date")
# df.shape

In [ ]:
# import local csv to colab
from google.colab import files
uploaded = files.upload()


In [ ]:
pltr_twitter_df=pltr_twitter_df.sort_values('Date')
pltr_twitter_df.head()

In [ ]:
 #Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1
    return result

In [ ]:
pltr_twtr_sentiments = []

for index, row in pltr_twitter_df.iterrows():
    try:
        text = row["Tweet"]
        date = row["Date"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        pol=  get_sentiment(compound)
        
        pltr_twtr_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu,
            "Polarity Score":pol
            
        })
        
    except AttributeError:
        pass



In [ ]:
pltr_twitter_sentiment_df = pd.DataFrame(pltr_twtr_sentiments)

pltr_twitter_sentiment_df=pltr_twitter_sentiment_df.sort_values('date')
pltr_twitter_sentiment_df=pltr_twitter_sentiment_df.set_index('date')
pltr_twitter_sentiment_df.head()

In [ ]:
# import pltr_twitter_sentiment_df csv to colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# Graph tallied Polarity Score 
pltr_twitter_sentiment_df.groupby('Polarity Score').size().plot(kind='bar')

## Pull data from Newsapi and run sentimment analysis

In [ ]:
pltr_headlines = newsapi.get_everything(
    q="PLTR",
    language="en",
    page_size=10,
    sort_by="relevancy",
    from_param="2021-02-29"
)

pltr_headlines

In [ ]:
 # Transform the response dictionary to a DataFrame
pltr_newsapi_df = pd.DataFrame.from_dict(pltr_headlines["articles"])
pltr_newsapi_df.head

In [ ]:
pltr_sentiments = []

for article in pltr_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        pltr_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

In [ ]:
# Create DataFrame
pltr_newsapi_sentiment_df = pd.DataFrame(pltr_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
pltr_newsapi_sentiment_df = pltr_newsapi_sentiment_df[cols]

pltr_sentiment_newsapi_df.sort_values('date')
pltr_sentiment_newsapi_df.set_index('date')
pltr_sentiment_newsapi_df.head()

In [ ]:
# Graph tallied Polarity Score 
pltr_sentiment_newsapi_df.groupby('Polarity Score').size().plot(kind='bar')

## WordCloud 

In [ ]:
df = pltr_twitter_sentiment_df["Polarity Score"]
#pltr_twitter_sentiment_df[pltr_twitter_sentiment_df["Polarity Score"]=='-1']
words = ' '.join(pltr_twitter_sentiment_df['text'])
cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])
stopwords = set(STOPWORDS)
stopwords.add("PLTR")
stopwords.add("Palantir")

wordcloud = WordCloud(stopwords=stopwords,
                      background_color='black',
                      width=3000,
                      height=2500
                     ).generate(cleaned_word)

In [ ]:
plt.figure(1,figsize=(12, 12))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
#Do we use cumulative return of compound or polarity?
c_return= (1 + pltr_twitter_sentiment_df['compound']).cumprod()
c_return.head()

pltr_twitter_sentiment_df.plot(kind='line', x='date', y='compound')

## Yahoo Finance Stock API 

In [ ]:
# Load PLTR stock closing prices from YFinance
pltr_yf = yf.download('PLTR',
                       start='2020-01-01',
                       end='2021-05-31',
                       progress=False,
                       index_col ='Date',
                       infer_datetime_format= True,
                       parse_dates = True,
                       interval='1d'

)
pltr_df = pd.DataFrame(pltr_yf)
pltr_df.head()



##Graph Sentiment score and Stock Price

## ML and LSTM Model 